### Imports

In [1]:
import pandas as pd
import numpy as np
import logging
import ipywidgets as widgets
from utils.postgre_conn import PostgresConnect
import plotly.graph_objects as go
from IPython.core.display import display
from re import search
from plotly.subplots import make_subplots
import plotly.express as px

### Connecting To Postgres

In [2]:
db_connection_dict = {    'drivername' : 'postgres',
    'host'      : '192.168.56.30',
    'port'      : '5432',
    'database'  : 'team_G_db',
    'username'  : 'dap',
    'password'  : 'dap'
}
postgres_conn = PostgresConnect(db_connection_dict)

### Visuals on Drug Label Data

In [ ]:
query = 'Select * from drug_labels'
final_drug  = postgres_conn.get_data(query)


#### Scatter Plot

In [ ]:
#Extracting top records which has a unique brand_names 

query = "select distinct(brand_name), manufacturer_name, count(brand_name), warning_s from drug_labels group by manufacturer_name, brand_name, warning_s order by count desc"
grouped_data = postgres_conn.get_data(query)
grouped_data = grouped_data.rename(columns = {0:'manufacturer_name', 1:'brand_name', 2:'Count', 3:'warning_s'})
print("In this we are visualizing the manufacturer_name in which there are brand_names and") 
print("the word count of the warning for each drug which is manufactured")
scatterplot = px.scatter(
    data_frame=grouped_data.head(20),
    x="warning_s",
    y="manufacturer_name",
    color="brand_name",                                       
    size = 'warning_s',
    size_max=40,   
    labels={
         "warning_s": "Number of words per warning",
         "manufacturer_name": "Name of Manufacturer",
         "brand_name": "Name of Brand"
     },
    title = "Warnings per Drug"
)

scatterplot.show()
    



#### Sunburts Plot

In [ ]:
if final_drug is not None:
    print("Visualizing the number of drugs which comes under the manufacturer name and when the pregnant women and children can use the drug")

    df=final_drug[final_drug["brand_name"].isin(['Mezereum','Ofloxacin','Sinus Severe','Naproxen'])]
    df=final_drug[final_drug["manufacturer_name"].isin(['Washington Homeopathic Products','A-S Medication Solutions','Walgreens'])]
    fig = px.sunburst(
        data_frame=df,
        path=["brand_name","pregnancy_or_breast_feeding","manufacturer_name","keep_out_of_reach_of_children"], #root,banches,leaves
        color="brand_name",
        color_discrete_sequence = px.colors.qualitative.Pastel,
        maxdepth = -1,
        title = "Count and Name of the Drug Used")
    fig.show()
else:
    print('Data Not found!!!')

### Scatter Plot for Drugs and recations combo

In [38]:
i = postgres_conn.get_data("select b.brand_name as drug1, e.brand_name as drug2, count(*) as usedtimes from drugs_event_lists b, drugs_event_lists e where  b.brand_name <> e.brand_name and b.safety_report_id = e.safety_report_id group by (b.brand_name, e.brand_name) having count(*) > 30 order by count(*) desc") 
print('''
* Flags all those combination of drug which have caused reactions in patients when consumed together,
taking threshold as minimum of 30 cases reported with the same combinations.''')
print(i.head(2))
len(i)


* Flags all those combination of drug which have caused reactions in patients when consumed together,
taking threshold as minimum of 30 cases reported with the same combinations.
      drug1     drug2  usedtimes
0   ADCIRCA  LETAIRIS        308
1  LETAIRIS   ADCIRCA        308


176

In [40]:
try:
    if i is not None:
        to_delete = []
        n = len(i)
        for j in range(0, n):
            if(n==88):
                break
            a = i.iloc[j]["drug1"]
            b = i.iloc[j]["drug2"]
            x = i[(i["drug1"]==b) & (i["drug2"]==a)].index
            i = i.drop(x)
            if len(x) > 0:
                n=n - 1
    else:
        print("No combinations found")
    drugs_combined = pd.DataFrame(i["drug1"]+','+i["drug2"])
    drugs_combined["Count"] = i["drug2"]
    drugs_combined=drugs_combined[0].reset_index(name="Drug_Combination")
    fig = px.scatter(drugs_combined.tail(15), x="index", y="Drug_Combination", labels = {"Drug_Combination":"DRUG COMBINATION", "index":"INDEX"},
        size="index", 
                 color="Drug_Combination", title="Drugs Combination causing reactions in patients when consumed together")
    fig.show()
    print('''The graph Flags the combination of drugs which have caused reactions in patients when consumed together, taking threshold as minimum of 30 cases reported with the same combinations''')
except:
    print('Unable to plot data for commonly reported drugs! ')

The graph Flags the combination of drugs which have caused reactions in patients when consumed together, taking threshold as minimum of 30 cases reported with the same combinations


### Drugs Recalled wrt to States and Classification

In [ ]:

query = 'SELECT country, classification, state, report_date FROM drugs_recalled;'
bar_line_df = postgres_conn.get_data(query)
try:
    if bar_line_df is not None:

        print('''
        The FDA classifies each recall in three classes, namely Class I, Class II, Class III, Class I being the most hazardous to public.
        To visualize the trends of we've tried to create a line chart to understand the trends.
        ''')

        figure = go.FigureWidget(layout = dict(title='Overall Recall Enforced Trend over the years'))
        for classification in bar_line_df['classification'].unique():
            temp_df = bar_line_df[(bar_line_df['classification']==classification)]
            values = temp_df['report_date'].value_counts().reset_index().sort_values(by='index')
            trace = go.Scatter(mode = 'lines', x = values['index'].astype('str'), y = values['report_date'], name=classification)
            figure.add_trace(trace)
        figure.update_layout(xaxis={'title':'Month Recall was Enforced'}, yaxis={'title':'Number of Dugs Recalled'})
        figure.show()
        print('''Insights:
    Annually an average of 1171.5 of recalls are enforced by FDA out if which 925.5 of them belong to Class II hazardous classification that is the drugs may have adversible health consequences.

    In March,2019 533 recalls were enforced by FDA out of which 471, nearly 89%(can be seen as a spike in red), were classified as Class I drugs i.e. the drug recalled had a high probability of causing adverse health consequences or even cause death.''')
except:
    print('Unable to create classification trend over the years.')

In [ ]:
print('''
The following visualizations are year wise distibution of the graph above. 
They are accompained by a visual for the year containing info for number of recalls enforced in each class by States in USA.
The graphs explains info regarding recalls by classification and state for a particular year. 
''')

if bar_line_df is not None:
    bar_line_df['year'] = bar_line_df['report_date'].dt.year
    bar_line_df['report_date'] = pd.to_datetime(bar_line_df['report_date'].dt.strftime('%Y-%m'))
    bar_line_df = bar_line_df[bar_line_df['country']=='United States']
    figure_dict = {}



    for year in sorted(bar_line_df['year'].unique()):
        figure = go.FigureWidget(layout = dict(width=400, height=400, title=str(year)))
        for classification in bar_line_df['classification'].unique():
            temp_df = bar_line_df[(bar_line_df['classification']==classification)& (bar_line_df['year']==year)]
            values = temp_df['report_date'].value_counts().reset_index().sort_values(by='index')
            trace = go.Scatter(mode = 'lines', x = values['index'].astype('str'), y = values['report_date'], name=classification)
            figure.add_trace(trace)
        figure.update_layout(xaxis={'title':'report_date'}, yaxis={'title':'Number of Dugs Recalled'})
        temp_df_2 = bar_line_df[bar_line_df['year']==year][['state', 'classification']].value_counts()
        temp_df_2 = temp_df_2.reset_index()
        temp_df_2.rename(columns = {0:'Drugs_Recalled'}, inplace=True)
        fig = px.bar(temp_df_2, x = 'classification', y='Drugs_Recalled',
                     color ='state', barmode='group',
                     width=400, height=400
                    )
        figure_2 = go.FigureWidget(fig)
        figure_dict[year] = [figure, figure_2]

    for year, figure_list in figure_dict.items():
        display(widgets.HBox(figure_list))

In [ ]:
print('''Insights:

1. In January 2014, 622 Class II drugs enforcement recalls were issued by FDA, which were nearly twice the enforcement issued in previous month.
2. In March 2019, it was North Carolina where FDA enforced 461 recalls of Class I hazardous drug.
3. There has been a decline of nearly 55%(2019-2020) in the enforcements issued by FDA.\n''')


### Creating TreeMap Graph to understand reason of Recall by each State

In [ ]:
query = 'SELECT country, state, reason_main, reason_description FROM drugs_recalled;'
drugs_df = postgres_conn.get_data(query)
print('''\nThe TreeMap shows Reasons for recall by each state, the size and color of the tile explains the number of recalls
enforced by FDA in that tile.
The treemap follows the following path:
Country-> State-> Reason Category-> Reason Description
''')

if drugs_df is not None:
    grouped_df = drugs_df.groupby(by=['country','state','reason_main', 'reason_description']).agg({'reason_main':'count'})
    grouped_df.rename(columns= {'reason_main':"Count"}, inplace=True)
    grouped_df.reset_index(inplace=True)
    grouped_df['reason'] = ['Reasons for Recall']*len(grouped_df)
    fig = px.treemap(grouped_df,
                     path=['reason','country','state','reason_main', 'reason_description'],
                     values='Count',
                     color='Count',
                     maxdepth=3,
                     color_continuous_scale = 'blues',
                     title='Reasons for Recall by States'
                    )
    fig.show()
print('''Insights:
1. For North Carolina, the reason for recall enforced by FDA in March 2019, was presence of MICROBIAL CONTAMINATION in drugs.

2. FDA has issued 466 enforcements of recall in Ontario, Canada because of the presence of Penicillin Cross Contamination in drugs.

3. Top 5 states where recalls were enforced:

    A. Florida, USA
    B. New Jersey, USA
    C. Illinois, USA
    D. Tennesse, USA
    E. California, USA

4. Top 5 reasons for recall were:

    A. Lack of Assurance of Sterility
    B. CGMP Deviations
    C. Labelling
    D. Penicillin Cross Contamination
    E. Microbial Contamination

''')

### Insight on Time taken by FDA to terminate report

In [ ]:
print('''
Since the initiation of recall reported by the Drugs, FDA takes signifcant amount of time to make sure that
everything is in accordance with Recall Enforcement Guidelines.
The below table shows minimum and maximum time taken by FDA by each classification to Terminate the Recall Report.
''')
query = 'SELECT classification, time_diff FROM drugs_recalled;'
time_diff_df = postgres_conn.get_data(query)
time_diff_df['time_diff'] = ((pd.to_timedelta(time_diff_df['time_diff']))/np.timedelta64(1, 'M')).round()
time_diff_df = time_diff_df[~(time_diff_df['classification']=='Not Yet Classified')]
display(pd.DataFrame(time_diff_df.groupby('classification')['time_diff'].max()))
display(pd.DataFrame(time_diff_df.groupby('classification')['time_diff'].min()))
print('''Insights:
1. Since the initiation of recall, it took a maximum of 90 months by FDA to ensure all the reasonable efforts have been made to remove the product or correct it in accordance with recall strategy.
2. Class II, III recalls can be terminated in span of a month as well.''')

### Combined Visualisations

In [ ]:
display(widgets.HTML('<h3><b>Combined Visualization</b></h3>'))
query = 'SELECT brand_name, recall_initiation_date, classification, status time_diff FROM drugs_recalled where brand_name IS NOT NULL;'
recall_enforced_brands = postgres_conn.get_data(query)

In [ ]:
query = 'SELECT * from drugs_event_lists'
drug_list = postgres_conn.get_data(query)

In [ ]:
result=pd.merge(drug_list, recall_enforced_brands,on="brand_name",how="inner")
result = pd.DataFrame(result)
result = result.groupby(["brand_name"]).first()
print('''
* Performing inner join on drug_list and drug_recall_dataframe on drug names
* Plotting the drugs which have been recalled before or after an event was registered to FDA.
* The first event reported by a patient has been taken for insight'''
     )

# taking the month difference on drug recalling and drug event being reported
result['time_delta'] = ((pd.to_datetime(result.recall_initiation_date) - pd.to_datetime(result.drug_event_date)))/np.timedelta64(1, 'M')
result['time_delta'] = result['time_delta'].astype(int)

result1=pd.merge(drug_list, recall_enforced_brands,on="brand_name",how="inner")
result1 = result1.drop_duplicates()

#Taking the count of events reported before a drug had been recalled.
result1=result1.groupby("brand_name").brand_name.count().sort_values(ascending=False).head(40).sort_index()
result1=pd.DataFrame(result1)
merged_df = pd.merge( result1,result, left_index=True,right_index=True, how='inner')


In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
print('''
Insights:
1. The Graph gives us the Drugs which were reported by a patient which were recalled successfully by FDA.
2. The months taken for FDA to recall a drug successfully by FDA
3. By this we can see a huge difference for a drug to be recalled, \n this is an indication for FDA to look into the Drug Events reported by public and other healthcare professionals'''
    )
print('''
1. The red line represents the number of times the drug was reported by public and health professionals before the recall was initiated
''')

# fig=go.Figure()
fig.add_trace(
        go.Bar(x = merged_df.index, y = merged_df["time_delta"], name='Months Taken'),secondary_y=False)
fig.add_trace(
        go.Scatter(x = merged_df.index, y = merged_df.brand_name, name='Cases Reported'), secondary_y=True)
fig.update_layout(
    title="Months taken to recall and Events Reported against the Drug",
    xaxis_title="Drug Names",
    yaxis_title="Months taken ",
    legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=13,
        color="Black"
    )
)
fig.update_yaxes(secondary_y=True, range = [merged_df["time_delta"].min(), merged_df["brand_name"].max()])
fig.show()

In [ ]:
# display(widgets.HTML('<h4><b>Months taken to recall and Events Reported against the Drug</b></h4>'))
fig = go.Figure().add_trace(go.Mesh3d(x=merged_df.index,
                   y=merged_df["time_delta"],
                   z=merged_df.brand_name,
                   opacity=0.5
                  ))
fig.update_layout(scene = dict(
                    xaxis_title='Brands',
                    yaxis_title='Months Taken',
                    zaxis_title='Cases Reported'),
                    width=600,
                    height=600,
                    margin=dict(r=20, b=10, l=10, t=10),
                  
                 )
fig.show()

In [ ]:
#Checking for the drugs listed above which have caused serious lifethreatning events by taking threshold as minimum of 1 case reported 

drugs_serious = postgres_conn.get_data("select b.brand_name from drugs_event_lists b where b.safety_report_id in (select safety_report_id from drugs_event where  seriousness_lifethreatening = '1' or seriousness_congenitalanomali = '1')group by b.brand_name having count(*) > 1 order by count(*) desc")
print('''\n\n*Selecting the drugs which are life threatning as reported by public or health professional\n from the drugs listed on the graph\n\n''')


#A list of life threatining drugs are seen below
drugs_serious.rename(columns = {0 : 'Drug_Name'}, inplace = True)


fig = go.Figure(data=[go.Table(header=dict(values=['BRAND_NAME']),
                 cells=dict(values=[drugs_serious.brand_name]))
                     ])

fig.show()
print('''The list of drug names which are life threatening as repoted by consumers''')


In [ ]:
drugs_serious_recalled = postgres_conn.get_data("select  b.brand_name, min(b.recall_initiation_date) as report, min(e.drug_event_date) as event from drugs_recalled b,drugs_event_lists e where b.brand_name in (select b.brand_name from drugs_event_lists b where b.safety_report_id in (select safety_report_id from drugs_event where  seriousness_lifethreatening = '1' or seriousness_congenitalanomali = '1')group by b.brand_name having count(*) > 5 order by count(*) desc) group by b.brand_name")
drugs_serious_recalled.rename(columns = {0 : 'Drug_Name', 'report' : 'Recall_DATE', 'event' : 'Event_DATE'}, inplace = True)
drugs_serious_recalled["Event_DATE"]=pd.to_datetime(drugs_serious_recalled["Event_DATE"], format='%Y-%m-%d')

In [ ]:
fig = go.Figure(data=[go.Table(header=dict(values=['BRAND_NAME','Recall_DATE','Event_DATE']),
                 cells=dict(values=[drugs_serious_recalled.brand_name, drugs_serious_recalled.Recall_DATE, drugs_serious_recalled.Event_DATE]))
                     ])
fig.show()
print('''INSIGHTS:\n''')
print('''*The following list of drugs shows thhe time difference btw the drug first reported and the drug recall initiation date\n''')
print('''*We can see that many drugs in the previous table has not been recalled yet.\n''')
print('''*This has to be taken care by FDA.''')